# Substitution Matrix Evaluation

This notebook evaluates amino acid substitution matrices based on their correlations with biochemical properties and codon distances. Statistical significance is assessed using FDR correction (Benjamini-Hochberg).

**Author:** KSS Project  
**Date:** 2025-01-19

## 1. Imports and Configuration

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy import stats
from itertools import combinations
from statsmodels.stats.multitest import multipletests

# Add src to path for main application modules
sys.path.append('../src')
from mutation_score import get_mutational_scores, load_substitution_matrix

# Import evaluation utilities from local module
from amino_acid_utils import (
    CODON_TABLE,
    AMINO_ACIDS,
    AA_3_TO_1,
    compute_codon_distance,
    get_min_codon_distance,
    calculate_property_distances,
    generate_amino_acid_pairs,
    precompute_codon_distances
)

## 2. Configuration Parameters

In [2]:
# Configuration
MATRICES_DIR = "../src/substitution_matrices"
PROPERTIES_FILE = "amino_acid_properties.csv"
OUTPUT_DIR = "matrix_evaluation_results"
TOP_N = 10

# Significance thresholds (FDR-corrected q-values)
SIGNIFICANCE_LEVELS = {
    0.001: "***",
    0.01: "**",
    0.05: "*"
}

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

## 3. Genetic Code and Amino Acid Data

In [3]:
# Use standard genetic code from amino_acid_utils
print(f"Amino acids: {len(AMINO_ACIDS)}")
print(f"Amino acids: {AMINO_ACIDS}")

Amino acids: 20
Amino acids: ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']


## 4. Helper Functions

In [4]:
def format_correlation(corr, qval):
    """Format correlation with significance stars."""
    formatted = f"{corr:.3f}"
    
    # Add significance stars based on q-value
    if qval < 0.001:
        formatted += "***"
    elif qval < 0.01:
        formatted += "**"
    elif qval < 0.05:
        formatted += "*"
    
    return formatted

## 5. Load Amino Acid Properties

In [5]:
# Load amino acid properties using mapping from amino_acid_utils
df_properties = pd.read_csv(PROPERTIES_FILE)
df_properties['AA_1letter'] = df_properties['AA'].map(AA_3_TO_1)
df_properties = df_properties.set_index('AA_1letter')
df_properties = df_properties.drop('AA', axis=1)

# Display properties
print("Amino acid properties:")
display(df_properties)

# Convert to dictionary format (using 1-letter codes)
amino_acid_properties = {col: df_properties[col].to_dict() for col in df_properties.columns}
property_names = list(amino_acid_properties.keys())

Amino acid properties:


,Hydrophob.,Polarity,Volume,Area Lost,pI,In-Out,Aroma.
AA_1letter,,,,,,,
A,0.31,8.1,27.5,0.74,6.00,0.20,0.025
R,-1.01,10.5,105.0,0.64,10.76,-1.34,0.750
N,-0.60,11.6,58.7,0.63,5.41,-0.69,0.200
D,-0.77,13.0,40.0,0.62,2.77,-0.72,-0.025
C,1.54,5.5,44.6,0.91,5.05,0.67,0.200
Q,-0.22,10.5,80.7,0.62,5.65,-0.74,0.300
E,-0.64,12.3,62.0,0.62,3.22,-1.09,0.050
G,0.00,9.0,53.2,0.72,5.97,0.06,0.000
H,0.13,10.4,79.0,0.78,7.59,0.04,0.450


## 6. Compute Reference Distances

In [6]:
# Generate all amino acid pairs using utility function
pairs = generate_amino_acid_pairs()
print(f"Total amino acid pairs: {len(pairs)}")

# Compute minimum codon distances using utility function
min_codon_distances = precompute_codon_distances()

# Display distribution
codon_dist_values = list(min_codon_distances.values())
print(f"\nMinimum codon distance distribution:")
print(pd.Series(codon_dist_values).value_counts().sort_index())

Total amino acid pairs: 190

Minimum codon distance distribution:
1     75
2    101
3     14
Name: count, dtype: int64


## 7. Load Available Matrices

In [7]:
# Find all matrix files (priority: .pkl > .json for precision)
matrix_files = {}

# First pass: load .pkl files (higher precision)
for file in os.listdir(MATRICES_DIR):
    if file.endswith('.pkl'):
        name = file[:-4]
        matrix_files[name] = os.path.join(MATRICES_DIR, file)

# Second pass: load .json files only if no .pkl exists
for file in os.listdir(MATRICES_DIR):
    if file.endswith('.json'):
        name = file[:-5]
        if name not in matrix_files:  # Only if no .pkl was found
            matrix_files[name] = os.path.join(MATRICES_DIR, file)

matrix_names = sorted(matrix_files.keys())
print(f"Available matrices: {len(matrix_names)}")
for name in matrix_names:
    ext = matrix_files[name].split('.')[-1]
    print(f"  - {name} (.{ext})")

Available matrices: 31
  - BENNER22 (.json)
  - BENNER6 (.json)
  - BENNER74 (.json)
  - BLASTP (.json)
  - BLOSUM45 (.json)
  - BLOSUM50 (.json)
  - BLOSUM62 (.json)
  - BLOSUM80 (.json)
  - BLOSUM90 (.json)
  - DAYHOFF (.json)
  - FENG (.json)
  - GENETIC (.json)
  - GONNET1992 (.json)
  - GRANTHAM (.json)
  - JOHNSON (.json)
  - JONES (.json)
  - LEVIN (.json)
  - MCLACHLAN (.json)
  - MDM78 (.json)
  - MIYATA (.json)
  - MIYATA_EVO (.pkl)
  - PAM120 (.json)
  - PAM250 (.json)
  - PAM30 (.json)
  - PAM70 (.json)
  - RAO (.json)
  - RISLER (.json)
  - SNEATH (.json)
  - STR (.json)
  - VTML160 (.json)
  - VTML250 (.json)


## 8. Evaluate Matrices

In [8]:
def evaluate_matrix(matrix_name):
    """Evaluate a single substitution matrix."""
    # Get mutation scores
    scores = get_mutational_scores(pairs, substitution_matrix_type=matrix_name)
    scores_array = np.array([scores[p] for p in pairs])
    
    # Calculate correlations with properties
    correlations = {}
    pvalues = {}
    
    for prop_name in property_names:
        prop_distances = calculate_property_distances(amino_acid_properties[prop_name], pairs)
        corr, pval = stats.spearmanr(prop_distances, scores_array)
        correlations[prop_name] = corr
        pvalues[prop_name] = pval
    
    # Calculate correlation with codon distances
    codon_dist_array = np.array([min_codon_distances[p] for p in pairs])
    corr_codon, pval_codon = stats.spearmanr(codon_dist_array, scores_array)
    correlations['MinCodD'] = corr_codon
    pvalues['MinCodD'] = pval_codon
    
    # FDR correction
    all_pvalues = [pvalues[prop] for prop in property_names] + [pval_codon]
    _, qvalues_array, _, _ = multipletests(all_pvalues, method='fdr_bh', alpha=0.05)
    
    qvalues = {k: q for k, q in zip(list(property_names) + ['MinCodD'], qvalues_array)}
    
    # Composite score
    composite_score = sum(correlations.values())
    
    return correlations, pvalues, qvalues, composite_score


# Evaluate all matrices
results = {}
print("Evaluating matrices...\n")

for i, matrix_name in enumerate(matrix_names, 1):
    try:
        correlations, pvalues, qvalues, composite = evaluate_matrix(matrix_name)
        results[matrix_name] = {
            'correlations': correlations,
            'pvalues': pvalues,
            'qvalues': qvalues,
            'composite_score': composite
        }
        print(f"[{i}/{len(matrix_names)}] {matrix_name}: {composite:.3f}")
    except Exception as e:
        print(f"[{i}/{len(matrix_names)}] {matrix_name}: ERROR - {e}")

print(f"\nSuccessfully evaluated: {len(results)}/{len(matrix_names)}")

Evaluating matrices...

[1/31] BENNER22: 3.439
[2/31] BENNER6: 2.982
[3/31] BENNER74: 3.441
[4/31] BLASTP: 3.187
[5/31] BLOSUM45: 2.953
[6/31] BLOSUM50: 3.142
[7/31] BLOSUM62: 3.271
[8/31] BLOSUM80: 3.227
[9/31] BLOSUM90: 3.202
[10/31] DAYHOFF: 3.012
[11/31] FENG: 3.525
[12/31] GENETIC: 2.158
[13/31] GONNET1992: 3.402
[14/31] GRANTHAM: 2.967
[15/31] JOHNSON: 2.878
[16/31] JONES: 3.309
[17/31] LEVIN: 2.787
[18/31] MCLACHLAN: 2.889
[19/31] MDM78: 3.054
[20/31] MIYATA: 3.566
[21/31] MIYATA_EVO: 4.578
[22/31] PAM120: 2.914
[23/31] PAM250: 3.013
[24/31] PAM30: 2.890
[25/31] PAM70: 2.935
[26/31] RAO: 2.480
[27/31] RISLER: 1.125
[28/31] SNEATH: 2.373
[29/31] STR: 2.650
[30/31] VTML160: 3.257
[31/31] VTML250: 3.351

Successfully evaluated: 31/31


## 9. Create Results Table

In [9]:
# Sort by composite score
sorted_matrices = sorted(results.items(), key=lambda x: x[1]['composite_score'], reverse=True)

# Build results dataframe
rows = []
for rank, (matrix_name, data) in enumerate(sorted_matrices, 1):
    row = {'Rank': rank, 'Matrix': matrix_name}
    
    # Add formatted correlations
    for prop in property_names + ['MinCodD']:
        corr = data['correlations'][prop]
        qval = data['qvalues'][prop]
        row[prop] = format_correlation(corr, qval)
    
    row['Composite'] = round(data['composite_score'], 3)
    rows.append(row)

df_results = pd.DataFrame(rows)

# Display top N
print(f"\n{'='*120}")
print(f"SUBSTITUTION MATRICES RANKING")
print(f"{'='*120}")
print("Significance: *** q<0.001, ** q<0.01, * q<0.05 (FDR-corrected)")
print(f"{'='*120}\n")
display(df_results.head(len(df_results)).style.hide(axis='index'))


SUBSTITUTION MATRICES RANKING
Significance: *** q<0.001, ** q<0.01, * q<0.05 (FDR-corrected)



Rank,Matrix,Hydrophob.,Polarity,Volume,Area Lost,pI,In-Out,Aroma.,MinCodD,Composite
1,MIYATA_EVO,0.849***,0.791***,0.286***,0.764***,0.419***,0.773***,0.284***,0.412***,4.578000
2,MIYATA,0.738***,0.727***,0.398***,0.554***,0.130,0.471***,0.277***,0.270***,3.566000
3,FENG,0.617***,0.628***,0.216**,0.524***,0.163*,0.416***,0.253***,0.707***,3.525000
4,BENNER74,0.697***,0.666***,0.302***,0.621***,0.006,0.433***,0.289***,0.427***,3.441000
5,BENNER22,0.674***,0.638***,0.283***,0.550***,0.031,0.397***,0.321***,0.544***,3.439000
6,GONNET1992,0.705***,0.670***,0.295***,0.629***,-0.001,0.440***,0.276***,0.388***,3.402000
7,VTML250,0.677***,0.638***,0.311***,0.595***,0.021,0.438***,0.302***,0.370***,3.351000
8,JONES,0.626***,0.600***,0.245***,0.503***,0.043,0.389***,0.303***,0.601***,3.309000
9,BLOSUM62,0.678***,0.660***,0.247***,0.592***,0.090,0.425***,0.240***,0.339***,3.271000
10,VTML160,0.638***,0.603***,0.306***,0.560***,0.030,0.412***,0.318***,0.392***,3.257000


## 10. Best Matrix Details

In [10]:
# Get best matrix
best_matrix_name = sorted_matrices[0][0]
best_data = sorted_matrices[0][1]

print(f"Best performing matrix: {best_matrix_name}")
print(f"Composite score: {best_data['composite_score']:.3f}\n")

# Detailed correlations
print("Detailed correlations:\n")
detail_rows = []
for prop in property_names + ['MinCodD']:
    corr = best_data['correlations'][prop]
    pval = best_data['pvalues'][prop]
    qval = best_data['qvalues'][prop]
    detail_rows.append({
        'Property': prop,
        'Correlation': round(corr, 4),
        'p-value': f"{pval:.2e}",
        'q-value': f"{qval:.2e}",
        'Significant': 'Yes' if qval < 0.05 else 'No'
    })

df_best = pd.DataFrame(detail_rows)
display(df_best)

Best performing matrix: MIYATA_EVO
Composite score: 4.578

Detailed correlations:



,Property,Correlation,p-value,q-value,Significant
0,Hydrophob.,0.8489,6.06e-54,4.85e-53,Yes
1,Polarity,0.7913,4.94e-42,1.98e-41,Yes
2,Volume,0.2860,6.34e-05,7.25e-05,Yes
3,Area Lost,0.7643,1.13e-37,2.27e-37,Yes
4,pI,0.4186,1.84e-09,2.95e-09,Yes
5,In-Out,0.7729,5.51e-39,1.47e-38,Yes
6,Aroma.,0.2835,7.36e-05,7.36e-05,Yes
7,MinCodD,0.4124,3.39e-09,4.51e-09,Yes


## 11. Save Results

In [11]:
# Save top N matrices
top_file = os.path.join(OUTPUT_DIR, f'substitution_matrices_ranking_top_{TOP_N}.csv')
df_results.head(TOP_N).to_csv(top_file, index=False)
print(f"Top {TOP_N} results saved to: {top_file}")

# Save complete ranking
complete_file = os.path.join(OUTPUT_DIR, 'substitution_matrices_ranking_complete.csv')
df_results.to_csv(complete_file, index=False)
print(f"Complete results saved to: {complete_file}")

# Save best matrix details
best_file = os.path.join(OUTPUT_DIR, f'{best_matrix_name}_detailed_correlations.csv')
df_best.to_csv(best_file, index=False)
print(f"Best matrix details saved to: {best_file}")

PermissionError: [Errno 13] Permission denied: 'matrix_evaluation_results\\substitution_matrices_ranking_top_10.csv'

## 12. Summary Statistics

In [ ]:
print("\nSUMMARY")
print("="*50)
print(f"Total matrices evaluated: {len(results)}")
print(f"Amino acid pairs analyzed: {len(pairs)}")
print(f"Properties analyzed: {len(property_names) + 1}")  # +1 for codon distance
print(f"\nBest matrix: {best_matrix_name}")
print(f"Best composite score: {best_data['composite_score']:.3f}")

# Count significant correlations
n_significant = sum(1 for q in best_data['qvalues'].values() if q < 0.05)
n_total = len(best_data['qvalues'])
print(f"Significant correlations (q < 0.05): {n_significant}/{n_total}")


SUMMARY
Total matrices evaluated: 32
Amino acid pairs analyzed: 190
Properties analyzed: 8

Best matrix: MIYATA_EVO
Best composite score: 4.578
Significant correlations (q < 0.05): 8/8
